In [6]:
from core.libs import Path, openpyxl, os, pd
from tqdm import tqdm

# Ruta base (ajústala según tu país/año/lote)
BASE_PATH = Path(r"C:\Users\HeyCe\World Tree Technologies Inc\Forest Inventory - Documentos\USA\2024_ForestInventoryQ1_25")

In [3]:
# Acumular errores
errores = []

# Buscar todos los .xlsx recursivamente
xlsx_files = list(BASE_PATH.rglob("*.xlsx"))

In [13]:
for file in tqdm(xlsx_files, desc="📂 Scanning OneDrive files"):
    try:
        wb = openpyxl.load_workbook(file, data_only=True)
        if "Summary" not in wb.sheetnames:
            errores.append((file, "❌ Missing 'Summary' sheet"))
            continue

        sheet = wb["Summary"]
        d3_value = sheet["D3"].value

        if not d3_value or not isinstance(d3_value, str) or d3_value.strip().lower() == "contract code":
            errores.append((file, f"⚠️ Invalid value in D3: '{d3_value}'"))

    except Exception as e:
        errores.append((file, f"💥 Error reading file: {e}"))



📂 Scanning OneDrive files:   0%|          | 0/64 [00:00<?, ?it/s]D:\MinicondaEnv\envs\WorldTreeSystem\lib\site-packages\openpyxl\reader\excel.py:237: UserWarning: Data Validation extension is not supported and will be removed
  ws_parser.bind_all()
D:\MinicondaEnv\envs\WorldTreeSystem\lib\site-packages\openpyxl\reader\excel.py:237: UserWarning: Conditional Formatting extension is not supported and will be removed
  ws_parser.bind_all()
📂 Scanning OneDrive files: 100%|██████████| 64/64 [00:21<00:00,  3.04it/s]


In [14]:
# Crear DataFrame
df = pd.DataFrame([
    {
        "Error": msg,
        "Filename": Path(path).name,
        "Directory Path": str(Path(path).parent)
    }
    for path, msg in errores
])

# Guardar como Excel
output_path = Path.cwd() / "malformed_d3_entries.xlsx"
df.to_excel(output_path, index=False)

print(f"\n✅ Report saved to: {output_path}")


✅ Report saved to: D:\LOCAL\WorldTreeSystem\AuxiliaryScripts\malformed_d3_entries.xlsx
